In [276]:
import pandas as pd
import bamboolib
import os
import datetime
import numpy as np
from matplotlib import pyplot
from matplotlib.ticker import MultipleLocator, FixedFormatter, FixedLocator
import plotly.express as px
import plotly.graph_objects as go


## Chemins des fichiers

In [2]:
#a changer localement
path_base="/home/aymeric/python-scripts/anses_medialab/datas/" #aymeric

file_segmentation = f"{path_base}segmentation_common_freq.csv"

index=f"{path_base}index_alldefault_nameexhaustive_aymeric.csv"

#all_corpus = f"{path_base}all_corpus_seg_with_sdhi_PI.csv"

#twittos = f"{path_base}tweets_pesticides/500_first_twittos_on_pesticides.csv"



## Chargement des données

In [253]:
#df0 = pd.read_csv(all_corpus, sep = "\t")
df_index = pd.read_csv(index, sep = "\t")
dfseg = pd.read_csv(file_segmentation, sep = "\t")

df_index['date'] = pd.to_datetime(df_index['date'], infer_datetime_format=True)
df_index['yearmonth']=(df_index['date'].dt.strftime('%Y-%m'))

df_index['date'] = df_index['date'].dt.date

## Fusion segment + indexation

In [ ]:
segment = dfseg[["yearmonth", "segm", "origin"]]


df0 = df_index.merge(segment, how = "inner", on = ["origin", "yearmonth"])
df0["start_segment"] = df0.groupby(['origin','segm'])["date"].transform('min')
df0["end_segment"] = df0.groupby(['origin','segm'])["date"].transform('max')


## Bump Chart

In [295]:
def ranked_element(data, which, top_rank):
    """
    data = the dataframe
    which = a string ; the variable that we want to rank and display on bump chart
    top_rank = an integer ; he rank that values should reach at least one time to be part of bump chart
    """
    
    df = (
        data
          .value_counts(["origin", which, 'segm', 'start_segment'])
          .groupby(["origin", "segm"])
          .rank("first", ascending=False)
          .rename("rank")
          .sort_index()
          .reset_index()
         )
    
    df.loc[df["rank"] > top_rank, f"top_rank_{top_rank}"] = 0
    df.loc[df["rank"] < top_rank, f"top_rank_{top_rank}"] = 1
    df["count_rank"] = df.groupby(["origin",which])[f"top_rank_{top_rank}"].transform("sum")
    df = df.loc[df["count_rank"] >= 1]
    
    return df
    


In [296]:
def bump_chart(data, which):
    
    """
    data = the dataframe
    """
    
    list_origin = ["media", "twitter", "facebook"]


    for n, x in enumerate(list_origin):
    
        df = data.loc[data["origin"]== x]
        n_top_ranked = 10
        top_sources = df[df["segm"] == df["segm"].max()].nsmallest(n_top_ranked, "rank")

        fig, ax = plt.subplots(figsize=(16, 10), subplot_kw=dict(ylim=(0.5, 0.5 + n_top_ranked)))
        
        fig.suptitle(x, fontsize=16)


        ax.xaxis.set_major_locator(MultipleLocator(1))
        ax.yaxis.set_major_locator(MultipleLocator(1))

        yax2 = ax.secondary_yaxis("right")
        yax2.yaxis.set_major_locator(FixedLocator(top_sources["rank"].to_list()))
        yax2.yaxis.set_major_formatter(FixedFormatter(top_sources[which].to_list()))


        for i, j in df.groupby([which]):
            ax = plt.plot("segm", "rank", "o-", data=j)
            

        plt.gca().invert_yaxis()
    
    return fig

In [305]:
def interactive_bump_chart(data, which):

    fig = px.line(df, x="start_segment", y="rank", color = which, facet_row="origin").update_traces(mode='markers+lines')
    fig['layout']['yaxis']['autorange'] = "reversed"

    fig.show()

In [309]:
df = ranked_element(data = df0, which="term", top_rank= 10)

In [310]:
bump = interactive_bump_chart(data = df, which = "term")